In [34]:
%load_ext autoreload
%autoreload 2

from semantic_parser.modules.verdant.actions import *
from semantic_parser.modules.verdant.openai_utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# FetchRelatedColumns
import os

client = OpenAIUtils(api_key=os.getenv("AZURE_OPENAI_API_KEY"))
db_manager = DatabaseManager(
    visible_tables=[
        "FundCashFlow",
        "FundInvestmentProperties",
        "FundInvestmentTimeProperties",
        "FundTimeProperties",
        "HandTransformedPortfolioInvestmentswithCFRaw"
    ]
)

fetch_column = FetchRelatedColumns(openai_client=client)
fetch_knowledge = FetchBackgroundKnowledge(openai_client=client)
sketch_sql = SketchSQL(openai_client=client)
generate_sql = GenerateSQL(openai_client=client)
execute_sql = ExecuteSQL(db_manager=db_manager)

In [28]:
def dict_to_string(d, separator=' '):
    """
    Flatten a nested dictionary of strings into a single string.
    
    Args:
        d: Dictionary (possibly nested) containing strings
        separator: String to join values with (default: ' ')
    
    Returns:
        Single concatenated string
    """
    result = []
    
    def flatten(obj):
        if isinstance(obj, dict):
            for value in obj.values():
                flatten(value)
        elif isinstance(obj, str):
            result.append(obj)
        elif isinstance(obj, (list, tuple)):
            for item in obj:
                flatten(item)
    
    flatten(d)
    return separator.join(result)

In [ ]:
result = {
    "query": "On a year-on-year basis (4Q24 vs 4Q23), which funds saw an uplift to overall MOIC (i.e., Fund MOIC) in 2024?"
}
output = fetch_column.execute(**result)
result = output.result
print(result)

{'approach': 'Derive each fund’s MOIC at the end of 4Q-2023 and at the end of 4Q-2024 from the quarterly investment-level table, then keep only those funds whose 2024 figure is higher than 2023. Return the fund identifiers (and optionally both MOIC figures and the uplift delta).', 'sketch': '1. From FundInvestmentTimeProperties keep only the two quarter-end records we care about – the rows whose quarter_date (or fiscal_quarter column) equals 2023-12-31 (4Q23) or 2024-12-31 (4Q24).\n2. Aggregate those rows to the fund level because each fund has many portfolio-company rows.  For each fund+quarter, add up cost_basis (capital_invested) and total_value (realised_value + unrealised_value) and then compute fund_MOIC = total_value / cost_basis.  If there is already a pre-calculated MOIC column, simply take MAX(MOIC) instead of re-computing.\n3. Pivot the two quarters into separate columns so that every fund is now one row that holds moic_4q23 and moic_4q24.  The easiest way is to self-join th

In [29]:
natural_language_sketch = dict_to_string(natural_language_sketch)
print(natural_language_sketch)

Compare fund-level MOIC at the end of 4Q24 with the value at the end of 4Q23 and return the funds whose MOIC increased. Step 1: Pull the position-level rows from FundInvestmentTimeProperties for quarter = '2024-12-31' (4Q24) and '2023-12-31' (4Q23). Step 2: For each fund in each quarter, aggregate to fund-level by summing TotalValue (realized + unrealized) and summing InvestedCost, then calculate MOIC = TotalValue / InvestedCost. Step 3: Place the two quarterly MOIC numbers side-by-side for every fund (self-join or conditional aggregation) so we have columns like moic_4q23 and moic_4q24. Step 4: Filter to funds where moic_4q24 > moic_4q23 (i.e., an uplift). Step 5: Return the fund identifier (and optionally the two MOIC values and the uplift amount/percentage) ordered by size of uplift or fund name. WHERE date filter for the two specific quarter-end dates GROUP BY fund_id, quarter to roll position rows up to fund level SUM for TotalValue and InvestedCost Computed column (TotalValue / I

In [33]:
kwargs_1 = {
    "natural_language_sketch": natural_language_sketch
}
output = generate_sql.execute(**kwargs_1)
sql_query = output.result['sql_query']
print(sql_query)

-- -------------------------------------------------------------------
--  Compare fund-level MOIC at 4Q23 vs 4Q24 and return the funds whose
--  MOIC improved.
-- -------------------------------------------------------------------
WITH quarterly_fund AS (
    /*-----------------------------------------------------------------
      Step 1 & 2 :  Pull position-level rows for the two quarter-ends,
      aggregate them to fund level and calculate MOIC.
      ----------------------------------------------------------------*/
    SELECT
        "FundID"                          AS fund_id,          -- fund identifier
        "QuarterEndDate"                 AS quarter_end_date, -- e.g. 2023-12-31, 2024-12-31
        SUM("TotalValue")   AS total_value,                   -- realised + unrealised
        SUM("InvestedCost") AS invested_cost,
        /* Protect against divide-by-zero */
        SUM("TotalValue") / NULLIF(SUM("InvestedCost"),0) AS moic
    FROM   "FundInvestmentTimeProperties"


In [32]:
output.result

{'sql_query': '-- Step-1 & 2 :  roll position rows up to fund-level for the two quarter-ends\nWITH fund_quarter_moic AS (\n    SELECT  \n        "FundID",                               -- fund identifier\n        "QuarterEndDate",                      -- quarter-end date (should be 2023-12-31 or 2024-12-31)\n        SUM("TotalValue")     AS total_value,  -- realised + unrealised value\n        SUM("InvestedCost")   AS invested_cost,\n        /* protect against divide-by-zero */\n        CASE WHEN SUM("InvestedCost") = 0 THEN NULL\n             ELSE SUM("TotalValue")::numeric / SUM("InvestedCost") END AS moic\n    FROM   "FundInvestmentTimeProperties"\n    WHERE  "QuarterEndDate" IN ( DATE \'2023-12-31\', DATE \'2024-12-31\')\n    GROUP  BY "FundID", "QuarterEndDate"\n),\n-- Step-3 : place 4Q23 & 4Q24 MOIC side-by-side\nmoic_pivot AS (\n    SELECT \n        q24."FundID",\n        q23.moic AS moic_4q23,\n        q24.moic AS moic_4q24,\n        (q24.moic - q23.moic)                       